# **run load_data.ipynb BEFORE running this!**

In [20]:
import pickle
import os.path
import numpy as np
normalized_data_file_name = 'data_normalized'
with open(normalized_data_file_name + '.pickle', 'rb') as handle:
    spectrum, temperature = pickle.load(handle)
    print(f"read data from {normalized_data_file_name}.pickle")
print(f"shape of spectrum data: {spectrum.shape}")
print(f"shape of temperature data: {temperature.shape}")
print()
print(f"there are {temperature.shape[0]} spectrums")
print(f"each spectrum is {spectrum.shape[1]} long, which is number of features")

read data from data_normalized.pickle
shape of spectrum data: (6000, 10000)
shape of temperature data: (6000, 1)

there are 6000 spectrums
each spectrum is 10000 long, which is number of features


In [13]:
indices_file_name = 'cross_validation_resample=2_fold=5'
with open(indices_file_name + '.pickle', 'rb') as handle:
    train_indices, test_indices = pickle.load(handle)
    print(f"got indices from {indices_file_name}.pickle")  
print()
print(f"sets of training indices: {len(train_indices)}")
print(f"number of training indices per set: {len(train_indices[0])}")
print(f"sets of testing indices: {len(test_indices)}")
print(f"number of testing indices per set: {len(test_indices[0])}")

got indices from cross_validation_resample=2_fold=5.pickle

sets of training indices: 10
number of training indices per set: 4800
sets of testing indices: 10
number of testing indices per set: 1200


In [14]:
input_dimension = spectrum.shape[1]
print(f"input dimension is: {input_dimension}")
output_dimension = temperature.shape[1]
print(f"output dimension is: {output_dimension}")

input dimension is: 10000
output dimension is: 1


In [15]:
import torch
import torch.nn as nn
class Model(torch.nn.Module):
    def __init__(self, device, input_dim=input_dimension):
        super().__init__()
        self.relu  = nn.ReLU()
        self.hidden_dim = 500
        self.linear1 = torch.nn.Linear(input_dim, self.hidden_dim)
        self.linear2 = torch.nn.Linear(self.hidden_dim, self.hidden_dim)
        self.linear3 = torch.nn.Linear(self.hidden_dim, 1)
        self.device = device
        self.to(device)
    def forward(self, x):
        y = self.linear3(self.relu(self.linear2(self.relu(self.linear1(x)))))
        # change: remove sigmoid, use y result as is
        # y = torch.sigmoid(y)
        return y

In [16]:
from torch.utils.data import DataLoader
from torch import optim
import numpy as np

loss_function = nn.MSELoss()

device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
class CalculateMSE():
    def __init__(self, net, n_epochs, batch_size):
        super().__init__()
        self.net = net
        #initialize some constants
        self.batch_size = 32
        self.learning_rate = 1e-4
        self.n_epochs = n_epochs
        self.net.apply(self.weights_init)   
    def weights_init(self,layer):
        if type(layer) == nn.Linear:
            nn.init.orthogonal_(layer.weight)
    def get_mse(self,train_data, train_label, test_data, test_label):
        train_set = torch.utils.data.TensorDataset(
            torch.Tensor(train_data).to(device), 
            torch.Tensor(train_label).to(device))
        val_set = torch.utils.data.TensorDataset(
            torch.Tensor(test_data).to(device), 
            torch.Tensor(test_label).to(device))
        loader_args = dict(batch_size=self.batch_size)
        train_loader = DataLoader(train_set, shuffle=True, drop_last=True, **loader_args)
        val_loader = DataLoader(val_set, shuffle=True, drop_last=True, **loader_args)
        train_loss = []
        validate_loss = []
        criterion = loss_function
        optimizer = optim.Adam(self.net.parameters(), lr=self.learning_rate) # weight_decay=0
        
        for epoch in range(0, self.n_epochs):
            # if epoch % 1000 == 0:
            #     print(f"epoch = {epoch}")
            epoch_train_loss=[]
            for i, data in enumerate(train_loader, 0):
                inputs, label = data
                y_pred = self.net(inputs.to(self.net.device))
                loss = criterion(y_pred, label.to(self.net.device))
                optimizer.zero_grad()
                loss.backward()
                optimizer.step()
                epoch_train_loss.append(loss.item())
            train_loss.append(np.mean(epoch_train_loss))
            # validation step: need to UN-normalize data!
            epoch_loss=[]

            for i, data in enumerate(val_loader, 0):
                with torch.no_grad():
                    inputs_validate, label_validate = data
                    y_pred_validate = self.net(inputs_validate.to(self.net.device))
                    loss_validate = criterion(y_pred_validate, label_validate.to(self.net.device)) # one number
                    epoch_loss.append(loss_validate.item())
            validate_loss.append(np.mean(epoch_loss))
        return np.min(validate_loss), self.net

In [17]:
from pathlib import Path
from datetime import datetime
n_epochs=1600
batch_size=32

PATH = 'model_dnn/'
Path(PATH).mkdir(parents=True, exist_ok=True)
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

# change: commented out: alraedy read in X, y data earlier
# response = pd.read_csv("1127_final_data/response.csv", header=None).values #input X
# spectra = pd.read_csv("1127_final_data/spectra.csv", header=None).values #ground truth label Y
# change: input dim = 1000
mdl = Model(device=device, input_dim=input_dimension)
losses = []
recovered_losses = []
print(f"number of epochs: {n_epochs}, batch size: {batch_size}, device: {mdl.device}")

f = open("log_dnn.txt", "w")
f.write("train START: " + datetime.now().strftime("%Y-%m-%d %H:%M:%S") + "\n")
f.write(f"number of epochs: {n_epochs}, batch size: {batch_size}, device: {device}\n")
for i, (train, test) in enumerate(zip(train_indices, test_indices)):
    print(f"we are on fold no.{i}")
    train_data, train_label= spectrum[train], temperature[train]
    test_data, test_label= spectrum[test], temperature[test]
    mse_calculator = CalculateMSE(mdl,n_epochs, batch_size)
    # print(f"train_data shape: {train_data.shape}")
    # print(f"train_label shape: {train_label.shape}")
    loss, model = mse_calculator.get_mse(train_data, 
                                        train_label, 
                                        test_data, 
                                        test_label)
    # want: display loss w.r.t pre-normalized (actual) data scale (during validation)
    losses.append(loss)
    print(f"\tloss: {loss}")
    print("\ttime: " + datetime.now().strftime("%Y-%m-%d %H:%M:%S"))
        
    f.write(f"fold = {i}")
    f.write(f"\tloss: {loss}\n")
    f.write("\ttime: " + datetime.now().strftime("%Y-%m-%d %H:%M:%S") +"\n")

    torch.save(model.state_dict(), PATH+'model_'+str(i))
f.write("train END: " + datetime.now().strftime("%Y-%m-%d %H:%M:%S") + "\n")
f.close()

number of epochs: 1, batch size: 32, device: cuda
we are on fold no.0
	loss: 0.18365182425524737
	time: 2023-05-20 19:43:35
we are on fold no.1
	loss: 0.20700642889415896
	time: 2023-05-20 19:43:36
we are on fold no.2
	loss: 0.18324313695366318
	time: 2023-05-20 19:43:37
we are on fold no.3
	loss: 0.310420638403377
	time: 2023-05-20 19:43:39
we are on fold no.4
	loss: 0.27960616148806905
	time: 2023-05-20 19:43:40
we are on fold no.5
	loss: 0.26498161417406957
	time: 2023-05-20 19:43:41
we are on fold no.6
	loss: 0.24355146470102104
	time: 2023-05-20 19:43:42
we are on fold no.7
	loss: 0.2159475313650595
	time: 2023-05-20 19:43:44
we are on fold no.8
	loss: 0.19530225243117358
	time: 2023-05-20 19:43:45
we are on fold no.9
	loss: 0.301129279104439
	time: 2023-05-20 19:43:46


In [18]:
losses_mean = np.mean(losses)
losses_std = np.std(losses)
print(f"mean losses: {losses_mean}, std: {losses_std}")

mean losses: 0.23848403317702785, std: 0.045759872739188655


In [19]:
number_figures = 10
import matplotlib.pyplot as plt

indices = torch.randint(0, len(spectrum),(number_figures,)).unique()
for i in indices:
    print(f"we use {i}th example")
    # change: cast i to int, since pandas not work with torch.int64
    # changed: removed figure, since the output is just one number
    spec = np.asarray(spectrum[int(i)]).flatten()
    temp = np.asarray(temperature[int(i)]).flatten()

    prediction = model(torch.Tensor(np.asarray(spectrum[int(i)])).to(model.device)).detach().cpu().flatten()
    
    prediction = prediction.item()
    ground_truth = temp.item()
    # recover data from normalization
    print(f"-----------------------------------------------------")
    print(f"\tthe prediction is: {prediction}")
    print(f"\tthe ground truth is: {ground_truth}")
    print(f"\tthe difference is: {prediction - ground_truth}")


we use 410th example
-----------------------------------------------------
	the prediction is: 21.067136764526367
	the ground truth is: 20.7
	the difference is: 0.3671367645263679
we use 534th example
-----------------------------------------------------
	the prediction is: 20.939960479736328
	the ground truth is: 20.7
	the difference is: 0.23996047973632884
we use 1238th example
-----------------------------------------------------
	the prediction is: 20.881813049316406
	the ground truth is: 21.0
	the difference is: -0.11818695068359375
we use 2005th example
-----------------------------------------------------
	the prediction is: 21.471771240234375
	the ground truth is: 21.2
	the difference is: 0.2717712402343757
we use 2451th example
-----------------------------------------------------
	the prediction is: 21.286157608032227
	the ground truth is: 21.2
	the difference is: 0.08615760803222727
we use 3101th example
-----------------------------------------------------
	the prediction i

# **skip cell 14 for now (it's in template file!)**